# Raster-based market access

Following in the footsteps of the Malaria Atlas Project's <a href="https://developers.google.com/earth-engine/datasets/catalog/Oxford_MAP_friction_surface_2015_v1_0">Global Friction Surface</a>, this notebook explores a set of tools for calculating:

1. Travel time rasters
2. Drive-time polygons
3. Marketsheds


In [1]:
import sys, os, importlib
import rasterio

import pandas as pd
import geopandas as gpd
import skimage.graph as graph

sys.path.append("../src")

import GOSTnetsraster.market_access as ma

%load_ext autoreload
%autoreload 2

In [2]:
results_folder = "tutorial_results"
if not os.path.exists(results_folder):
    os.makedirs(results_folder)
tutorial_folder = "tutorial_data"
dests = os.path.join(tutorial_folder, "SEN_cities.geojson")
friction_surface = os.path.join(tutorial_folder, "SEN_friction_surface.tif")

inD = gpd.read_file(dests)
inR = rasterio.open(friction_surface)
inD = inD.to_crs(inR.crs) # destinations and raster need to be in the same CRS

frictionD = inR.read()[0,:,:]
# convert friction surface to traversal time (lazily). Original data are
#    the original data are minutes to travel 1 m, so we will convert to 
#    minutes to cross the cell
frictionD = frictionD * 1000
mcp = graph.MCP_Geometric(frictionD)

In [3]:
ma.calculate_travel_time?

Signature: ma.calculate_travel_time(inH, mcp, destinations, out_raster='')
Docstring:
Calculate travel time raster

INPUTS
    inH [rasterio object] - template raster used to identify locations of destinations
    mcp [skimage.graph.MCP_Geometric] - input graph
    destinations [geopandas df] - destinations for nearest calculations
    
LINKS
    https://scikit-image.org/docs/0.7.0/api/skimage.graph.mcp.html#skimage.graph.mcp.MCP.find_costs
File:      c:\wbg\work\code\gostnetsraster\src\gostnetsraster\market_access.py
Type:      function

In [4]:
travel_costs, traceback = ma.calculate_travel_time(inR, mcp, inD)
travel_costs = travel_costs.astype(inR.meta['dtype'])
with rasterio.open(os.path.join(results_folder, "least_cost_travel_time.tif"), 'w', **inR.meta) as out_file:
    out_file.write_band(1, travel_costs)

In [5]:
ma.generate_feature_vectors?

Signature:
ma.generate_feature_vectors(
    network_r,
    mcp,
    inH,
    threshold,
    featIdx='tempID',
    verbose=True,
)
Docstring:
Generate individual market sheds for each feature in the input dataset

INPUTS
    network_r [rasterio] - raster from which to grab index for calculations in MCP
    mcp [skimage.graph.MCP_Geometric] - input graph
    inH [geopandas data frame] - geopandas data frame from which to calculate features
    threshold [list of int] - travel treshold from which to calculate vectors in units of graph
    featIdx [string] - column name in inH to append to output marketshed dataset. 'tempID' for default.
    
RETURNS
    [geopandas dataframe]
File:      c:\wbg\work\code\gostnetsraster\src\gostnetsraster\market_access.py
Type:      function

In [6]:
drive_time_thresholds = [60, 120, 180, 240] # minutes
drive_vectors = ma.generate_feature_vectors(inR, mcp, inD, drive_time_thresholds)
drive_vectors.to_file(os.path.join(results_folder, "drive_vectors.shp"))

12:06:30	1 of 30: 0
12:06:30	2 of 30: 1
12:06:31	3 of 30: 2
12:06:31	4 of 30: 3
12:06:31	5 of 30: 4
12:06:31	6 of 30: 5
12:06:32	7 of 30: 6
12:06:32	8 of 30: 7
12:06:32	9 of 30: 8
12:06:32	10 of 30: 9
12:06:33	11 of 30: 10
12:06:33	12 of 30: 11
12:06:33	13 of 30: 12
12:06:34	14 of 30: 13
12:06:34	15 of 30: 14
12:06:34	16 of 30: 15
12:06:34	17 of 30: 16
12:06:35	18 of 30: 17
12:06:35	19 of 30: 18
12:06:35	20 of 30: 19
12:06:36	21 of 30: 20
12:06:36	22 of 30: 21
12:06:36	23 of 30: 22
12:06:36	24 of 30: 23
12:06:37	25 of 30: 24
12:06:37	26 of 30: 25
12:06:37	27 of 30: 26
12:06:38	28 of 30: 27
12:06:38	29 of 30: 28
12:06:38	30 of 30: 29


In [7]:
ma.calculate_gravity?

Signature:
ma.calculate_gravity(
    inH,
    mcp,
    dests,
    gravity_col,
    outfile='',
    decayVals=[0.01, 0.005, 0.001, 0.0007701635, 0.0003850818, 0.0001925409, 9.62704e-05, 3.85082e-05, 1e-05],
)
Docstring:
Using a friction surface, run a gravity model to evaluate access to all cities

Parameters
----------
inH : rasterio object
    rasterio object of friction surface
mcp : skimage.graph.MCP_Geometric
    graph used to calculate travel times; must match shape of inH
dests : geopandas dataframe
    Destination coordinates for gravity model, must be in same CRS as inH
gravity_col : string
    column in dests that describes the attractiveness of the destination
outfile : string, optional
    Path to save the gravity model output. Defaults to '' which writes nothing
decayVals : list, optional
    List of decay values for market access. Defaults to a predefined list.

Returns
-------
final_gravity : numpy array
    3D array of gravity model results, with the third dimension corr

In [9]:
# calculate gravity to nearest cities, using the population column GC_POP_TOT_2025
gravity_col = "GC_POP_TOT_2025"
gravity_file = os.path.join(results_folder, f"gravity_cities_{gravity_col}.tif")
gravity = ma.calculate_gravity(inR, mcp, inD, gravity_col, outfile = gravity_file)